<a href="https://colab.research.google.com/github/gc2321/3546-Deep-Learning/blob/main/assign_2/assign_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Flowers using Transfer Learning in Keras

Assignment 2 - Convolutional Neural Nets and Transfer Learning

By Gordon Chan (qq525548)


In [5]:
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
random.seed(42)


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
data_dir = "/content/gdrive/MyDrive/3546/assign_2/flower_photos"

In [12]:
os.listdir(data_dir)

['tulips', 'roses', 'sunflowers', 'dandelion', 'daisy']

## Loading data

In [13]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define image size and batch size, resize to dimension expected by VGG16
img_height, img_width = 224, 224
batch_size = 32

# Load the dataset from the directory
full_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,  # 20% for testing
    subset="both",  # Create both training and validation subsets
    seed=42,  # Seed for reproducibility
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Separate the datasets
train_dataset = full_dataset[0]
test_dataset = full_dataset[1]


Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Using 734 files for validation.


In [29]:
class_names = train_dataset.class_names

In [30]:
# Count the number of images in each class in the training and testing sets
train_class_counts = {class_name: 0 for class_name in class_names}
test_class_counts = {class_name: 0 for class_name in class_names}

for images, labels in train_dataset:
    for label in labels.numpy():
        train_class_counts[class_names[label]] += 1

for images, labels in test_dataset:
    for label in labels.numpy():
        test_class_counts[class_names[label]] += 1

print("Training set class counts:", train_class_counts)
print("Testing set class counts:", test_class_counts)

KeyboardInterrupt: 